In [42]:
import torch
from torch import nn

In [43]:
print(torch.cuda.is_available())

True


In [44]:
class AlexNet(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,padding=2),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Conv2d(256,384,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(384,384,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(384,256,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )

    def forward(self, x):
        x = self.net(x) 
        x = x.view(x.size(0),-1)
        return self.classifier(x)

In [45]:
import torchvision
from torchvision.transforms import transforms

transform = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [46]:
import torch.optim as optim

learning_rate=0.01
loss_function=nn.CrossEntropyLoss().to('cuda')
model = AlexNet(10).to('cuda')
optimizer=optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9,weight_decay=0.0005)
num_epochs=10


In [47]:
for epochs in range(num_epochs):
    running_loss=0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_function(outputs,labels)

        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
    print(f"The loss after {epochs+1} epochs is {running_loss}")
    running_loss=0.0
print("Training Finished")

The loss after 1 epochs is 900.4106590747833
The loss after 2 epochs is 899.7536218166351
The loss after 3 epochs is 803.653507232666
The loss after 4 epochs is 700.2625815868378
The loss after 5 epochs is 612.0039924383163
The loss after 6 epochs is 530.3300919532776
The loss after 7 epochs is 469.6639911532402
The loss after 8 epochs is 406.09337478876114
The loss after 9 epochs is 356.35439813137054
The loss after 10 epochs is 314.08887046575546
Training Finished


In [48]:
from sklearn.metrics import accuracy_score
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = accuracy_score(all_labels, all_predictions)
print(accuracy)


0.727
